# Object Designator
Object designators are used to describe objects located in the BulletWorld or the real environment and then resolve them during runtime to concrete objects.

Object designators are different from the Object class in bullet_world.py in the way that they just describe an object and do not create objects or provide methods to manipulate them. Nevertheless, object designators contain a reference to the BulletWorld object.

An Object designator takes two parameters, of which at least one has to be provided. These parameters are:

   * A list of names 
   * A list of types 
   
Object Designators work similar to Location designators, they get constrains describing a set of objects and when resolved return a specific instance. 

For all following examples we need a BulletWorld, so let's create one.

In [1]:
from pycram.bullet_world import BulletWorld, Object
from pycram.enums import ObjectType
from pycram.pose import Pose

world = BulletWorld()

Unknown tag "material" in /robot[@name='plane']/link[@name='planeLink']/collision[1]
Unknown tag "contact" in /robot[@name='plane']/link[@name='planeLink']
Unknown tag "material" in /robot[@name='plane']/link[@name='planeLink']/collision[1]
Unknown tag "contact" in /robot[@name='plane']/link[@name='planeLink']


In [7]:
world.exit()

## Believe Object
This object designator is used to describe objects that are located in the BulletWorld. So objects that are in the belief state, hence the name. In the future when there is a perception interface, there will be a ```RealObject``` description which will be used to describe objects in the real world. 

Since ```BelieveObject``` describes Objects in the BulletWorld we create a few.

In [2]:
kitchen = Object("kitchen", ObjectType.ENVIRONMENT, "kitchen.urdf")
milk = Object("milk", ObjectType.MILK, "milk.stl", pose=Pose([1.3, 1, 0.9]))
cereal = Object("froot_loops", ObjectType.BREAKFAST_CEREAL, "breakfast_cereal.stl", pose=Pose([1.3, 0.9, 0.95]))
spoon = Object("spoon", ObjectType.SPOON, "spoon.stl", pose=Pose([1.3, 1.1, 0.87]))

Scalar element defined multiple times: limit
Scalar element defined multiple times: limit


Now that we have objects we can create an object designator to describe them. For the start we want an object designator only describing the milk. Since all objects have unique names we can create an object designator using a list with only the name of the object. 

In [3]:
from pycram.designators.object_designator import BelieveObject

object_description = BelieveObject(names=["milk"])

print(object_description.resolve())

BelieveObject.Object(name='milk', type=<ObjectType.MILK: 1>, bullet_world_object=Object(world=<pycram.bullet_world.BulletWorld object at 0x7fd84c0f46a0>, 
local_transformer=<pycram.local_transformer.LocalTransformer object at 0x7fd84c0f4f70>, 
name=milk, 
type=ObjectType.MILK, 
color=[1, 1, 1, 1], 
id=3, 
path=/home/jdech/workspace/ros/src/pycram-1/src/pycram/../../resources/cached/milk.urdf, 
joints: ..., 
links: ..., 
attachments: ..., 
cids: ..., 
original_pose=header: 
  seq: 0
  stamp: 
    secs: 1699448681
    nsecs: 742550373
  frame_id: "map"
pose: 
  position: 
    x: 1.3
    y: 1.0
    z: 0.9
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.0
    w: 1.0, 
tf_frame=milk_3, 
urdf_object: ..., 
_current_pose=header: 
  seq: 0
  stamp: 
    secs: 1699448681
    nsecs: 742550373
  frame_id: "map"
pose: 
  position: 
    x: 1.3
    y: 1.0
    z: 0.9
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.0
    w: 1.0, 
_current_link_poses={'milk_main': header: 
  seq: 0
  stamp: 
    secs: 16

You can also use the type to describe objects, so now we want to have an object designator that describes every food in the world. 

In [4]:
from pycram.designators.object_designator import BelieveObject

object_description = BelieveObject(types=[ObjectType.MILK, ObjectType.BREAKFAST_CEREAL])

print(object_description.resolve())

BelieveObject.Object(name='milk', type=<ObjectType.MILK: 1>, bullet_world_object=Object(world=<pycram.bullet_world.BulletWorld object at 0x7fd84c0f46a0>, 
local_transformer=<pycram.local_transformer.LocalTransformer object at 0x7fd84c0f4f70>, 
name=milk, 
type=ObjectType.MILK, 
color=[1, 1, 1, 1], 
id=3, 
path=/home/jdech/workspace/ros/src/pycram-1/src/pycram/../../resources/cached/milk.urdf, 
joints: ..., 
links: ..., 
attachments: ..., 
cids: ..., 
original_pose=header: 
  seq: 0
  stamp: 
    secs: 1699448681
    nsecs: 742550373
  frame_id: "map"
pose: 
  position: 
    x: 1.3
    y: 1.0
    z: 0.9
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.0
    w: 1.0, 
tf_frame=milk_3, 
urdf_object: ..., 
_current_pose=header: 
  seq: 0
  stamp: 
    secs: 1699448681
    nsecs: 742550373
  frame_id: "map"
pose: 
  position: 
    x: 1.3
    y: 1.0
    z: 0.9
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.0
    w: 1.0, 
_current_link_poses={'milk_main': header: 
  seq: 0
  stamp: 
    secs: 16

## Object Part 
Part of object designators can be used to describe something as part of another object. For example, you could describe a specific drawer as part of the kitchen. This is necessary since the drawer is no single BulletWorld Object but rather a link of the kitchen which is a BulletWorld Object.

For this example we need just need the kitchen, if you didn't spawn it in the previous example you can spawn it with the following cell.

In [ ]:
kitchen = Object("kitchen", ObjectType.ENVIRONMENT, "kitchen.urdf")

In [ ]:
from pycram.designators.object_designator import ObjectPart, BelieveObject

kitchen_desig = BelieveObject(names=["kitchen"]).resolve()

object_description = ObjectPart(names=["sink_area_left_upper_drawer_main"], part_of=kitchen_desig)

print(object_description.resolve())

## Object Designators as Generators 
Similar to location designators object designators can be used as generators to iterate through every object that they are describing. We will see this at the example of an object designator describing every type of food. 

For this we need some objects, so if you didn't already spawn them you can use the next cell for this.

In [ ]:
kitchen = Object("kitchen", ObjectType.ENVIRONMENT, "kitchen.urdf")
milk = Object("milk", ObjectType.MILK, "milk.stl", pose=Pose([1.3, 1, 0.9]))
cereal = Object("froot_loops", ObjectType.BREAKFAST_CEREAL, "breakfast_cereal.stl", pose=Pose([1.3, 0.9, 0.95]))
spoon = Object("spoon", ObjectType.SPOON, "spoon.stl", pose=Pose([1.3, 1.1, 0.87]))

In [6]:
from pycram.designators.object_designator import BelieveObject

object_description = BelieveObject(types=[ObjectType.MILK, ObjectType.BREAKFAST_CEREAL])

for obj in object_description:
    print(obj, "\n")

BelieveObject.Object(name='milk', type=<ObjectType.MILK: 1>, bullet_world_object=Object(world=<pycram.bullet_world.BulletWorld object at 0x7fd84c0f46a0>, 
local_transformer=<pycram.local_transformer.LocalTransformer object at 0x7fd84c0f4f70>, 
name=milk, 
type=ObjectType.MILK, 
color=[1, 1, 1, 1], 
id=3, 
path=/home/jdech/workspace/ros/src/pycram-1/src/pycram/../../resources/cached/milk.urdf, 
joints: ..., 
links: ..., 
attachments: ..., 
cids: ..., 
original_pose=header: 
  seq: 0
  stamp: 
    secs: 1699448681
    nsecs: 742550373
  frame_id: "map"
pose: 
  position: 
    x: 1.3
    y: 1.0
    z: 0.9
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.0
    w: 1.0, 
tf_frame=milk_3, 
urdf_object: ..., 
_current_pose=header: 
  seq: 0
  stamp: 
    secs: 1699448681
    nsecs: 742550373
  frame_id: "map"
pose: 
  position: 
    x: 1.3
    y: 1.0
    z: 0.9
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.0
    w: 1.0, 
_current_link_poses={'milk_main': header: 
  seq: 0
  stamp: 
    secs: 16